# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 6 2023 9:44AM,258692,15,9233904,"Brookfield Pharmaceuticals, LLC",Released
1,Mar 6 2023 9:44AM,258692,15,9233918,"Brookfield Pharmaceuticals, LLC",Released
2,Mar 6 2023 9:44AM,258692,15,9233919,"Brookfield Pharmaceuticals, LLC",Released
3,Mar 6 2023 9:44AM,258692,15,9233920,"Brookfield Pharmaceuticals, LLC",Released
4,Mar 6 2023 9:44AM,258692,15,9233921,"Brookfield Pharmaceuticals, LLC",Released
5,Mar 6 2023 9:44AM,258692,15,9233922,"Brookfield Pharmaceuticals, LLC",Released
6,Mar 6 2023 9:44AM,258692,15,9233940,"Brookfield Pharmaceuticals, LLC",Released
7,Mar 6 2023 9:44AM,258692,15,9233941,"Brookfield Pharmaceuticals, LLC",Released
8,Mar 6 2023 9:44AM,258692,15,9233942,"Brookfield Pharmaceuticals, LLC",Released
9,Mar 6 2023 9:44AM,258692,15,9233949,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,258685,Released,3
13,258686,Released,1
14,258688,Released,1
15,258690,Released,20
16,258692,Released,24


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258685,32.0,3.0
258686,NaN,1.0
258688,NaN,1.0
258690,NaN,20.0
258692,NaN,24.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258664,2.0,3.0
258668,0.0,1.0
258670,0.0,1.0
258671,12.0,2.0
258672,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258664,2,3
258668,0,1
258670,0,1
258671,12,2
258672,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258664,2,3
1,258668,0,1
2,258670,0,1
3,258671,12,2
4,258672,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258664,2,3
1,258668,,1
2,258670,,1
3,258671,12,2
4,258672,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 6 2023 9:44AM,258692,15,"Brookfield Pharmaceuticals, LLC"
24,Mar 6 2023 9:42AM,258690,15,"Alliance Pharma, Inc."
44,Mar 6 2023 9:18AM,258688,10,ISDIN Corporation
45,Mar 6 2023 9:13AM,258685,10,"Methapharm, Inc."
80,Mar 6 2023 9:08AM,258686,18,"Emersa Waterbox, LLC"
81,Mar 6 2023 9:04AM,258684,18,"Emersa Waterbox, LLC"
82,Mar 6 2023 8:58AM,258681,10,"Methapharm, Inc."
94,Mar 6 2023 8:57AM,258680,18,"Emersa Waterbox, LLC"
95,Mar 6 2023 8:57AM,258671,10,Bio-PRF
109,Mar 6 2023 8:46AM,258672,18,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 6 2023 9:44AM,258692,15,"Brookfield Pharmaceuticals, LLC",,24
1,Mar 6 2023 9:42AM,258690,15,"Alliance Pharma, Inc.",,20
2,Mar 6 2023 9:18AM,258688,10,ISDIN Corporation,,1
3,Mar 6 2023 9:13AM,258685,10,"Methapharm, Inc.",32,3
4,Mar 6 2023 9:08AM,258686,18,"Emersa Waterbox, LLC",,1
5,Mar 6 2023 9:04AM,258684,18,"Emersa Waterbox, LLC",,1
6,Mar 6 2023 8:58AM,258681,10,"Methapharm, Inc.",,12
7,Mar 6 2023 8:57AM,258680,18,"Emersa Waterbox, LLC",,1
8,Mar 6 2023 8:57AM,258671,10,Bio-PRF,12,2
9,Mar 6 2023 8:46AM,258672,18,"Emersa Waterbox, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 9:44AM,258692,15,"Brookfield Pharmaceuticals, LLC",24,
1,Mar 6 2023 9:42AM,258690,15,"Alliance Pharma, Inc.",20,
2,Mar 6 2023 9:18AM,258688,10,ISDIN Corporation,1,
3,Mar 6 2023 9:13AM,258685,10,"Methapharm, Inc.",3,32
4,Mar 6 2023 9:08AM,258686,18,"Emersa Waterbox, LLC",1,
5,Mar 6 2023 9:04AM,258684,18,"Emersa Waterbox, LLC",1,
6,Mar 6 2023 8:58AM,258681,10,"Methapharm, Inc.",12,
7,Mar 6 2023 8:57AM,258680,18,"Emersa Waterbox, LLC",1,
8,Mar 6 2023 8:57AM,258671,10,Bio-PRF,2,12
9,Mar 6 2023 8:46AM,258672,18,"Emersa Waterbox, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 9:44AM,258692,15,"Brookfield Pharmaceuticals, LLC",24,
1,Mar 6 2023 9:42AM,258690,15,"Alliance Pharma, Inc.",20,
2,Mar 6 2023 9:18AM,258688,10,ISDIN Corporation,1,
3,Mar 6 2023 9:13AM,258685,10,"Methapharm, Inc.",3,32
4,Mar 6 2023 9:08AM,258686,18,"Emersa Waterbox, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 9:44AM,258692,15,"Brookfield Pharmaceuticals, LLC",24,NaN
1,Mar 6 2023 9:42AM,258690,15,"Alliance Pharma, Inc.",20,NaN
2,Mar 6 2023 9:18AM,258688,10,ISDIN Corporation,1,NaN
3,Mar 6 2023 9:13AM,258685,10,"Methapharm, Inc.",3,32.0
4,Mar 6 2023 9:08AM,258686,18,"Emersa Waterbox, LLC",1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 6 2023 9:44AM,258692,15,"Brookfield Pharmaceuticals, LLC",24,0.0
1,Mar 6 2023 9:42AM,258690,15,"Alliance Pharma, Inc.",20,0.0
2,Mar 6 2023 9:18AM,258688,10,ISDIN Corporation,1,0.0
3,Mar 6 2023 9:13AM,258685,10,"Methapharm, Inc.",3,32.0
4,Mar 6 2023 9:08AM,258686,18,"Emersa Waterbox, LLC",1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1810736,22,44.0
15,517382,44,0.0
18,1034722,4,0.0
19,258664,3,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1810736,22,44.0
1,15,517382,44,0.0
2,18,1034722,4,0.0
3,19,258664,3,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,22,44.0
1,15,44,0.0
2,18,4,0.0
3,19,3,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,22.0
1,15,Released,44.0
2,18,Released,4.0
3,19,Released,3.0
4,10,Executing,44.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19
Status,,,,
Executing,44.0,0.0,0.0,2.0
Released,22.0,44.0,4.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19
0,Executing,44.0,0.0,0.0,2.0
1,Released,22.0,44.0,4.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19
0,Executing,44.0,0.0,0.0,2.0
1,Released,22.0,44.0,4.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()